In [3]:
import os

len(os.listdir('C:/Users/vian8/Desktop/Tugas2/SNAPGRADE/inputs/cross_data_raw'))

101

In [ ]:
directory = 'C:/Users/vian8/Desktop/Tugas2/SNAPGRADE/inputs/cross_data_raw'

files = os.listdir(directory)

image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

for i, filename in enumerate(image_files, start=1):
    old_path = os.path.join(directory, filename)
    file_extension = os.path.splitext(filename)[1]
    new_path = os.path.join(directory, f"{i}{file_extension}")
    os.rename(old_path, new_path)

print("Files have been renamed successfully.")

Files have been renamed successfully.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def resize_image(image, target_size=(800, 1000)):
  '''Resize the image while maintaining aspect ratio'''
  h, w = image.shape[:2]
  scale = min(target_size[1] / w, target_size[0] / h)
  new_w = int(w * scale)
  new_h = int(h * scale)
  resized_image = cv2.resize(image, (new_w, new_h))
  return resized_image

def logarithmic_transformation(image, epsilon=1e-5):
  '''Apply logarithmic transformation to the image with zero value handling'''
  c = 255 / np.log(1 + np.max(image))
  # Epsilon zero-handling technique
  log_image = c * (np.log(1 + image + epsilon))
  log_image = np.array(log_image, dtype=np.uint8)

  return log_image

def contrast_stretching(image):
  min_val = np.min(image)
  max_val = np.max(image)
  stretched = (image - min_val) * (255 / (max_val - min_val))
  return stretched.astype(np.uint8)

def gaussian_blur(image, mode='Soft'):
  if mode == 'Soft':
    kernel_size = (3,3)
  elif mode == 'Medium':
    kernel_size = (5,5)
  elif mode == 'Hard':
    kernel_size = (7,7)
  else:
    raise ValueError("Mode must be 'Soft', 'Medium', or 'Hard'")

  return cv2.GaussianBlur(image, kernel_size, 0)

def measure_blurriness(image):
  # Apply the Laplacian operator to detect edges
  laplacian = cv2.Laplacian(image, cv2.CV_64F)
  # Variance of Laplacian
  variance = laplacian.var()

  return variance

def adaptive_gaussian_blur(image, desired_blur=100, max_iterations=100):
  # Measure initial blur level
  initial_blur = measure_blurriness(image)

  # Set a starting kernel size
  kernel_size = 5

  for iteration in range(max_iterations):
      # Apply Gaussian blur with the current kernel size
      blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

      # Measure the blur after applying Gaussian blur
      current_blur = measure_blurriness(blurred_image)

      # If the current blur exceeds the desired blur, stop
      if current_blur > desired_blur:
          kernel_size += 2
      else:
        break

  final_blurred_img = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
  final_blur = measure_blurriness(final_blurred_img)

  print(f"Initial Blur: {initial_blur}, Final Blur: {final_blur}, Kernel Size: {kernel_size}, Iterations: {iteration+1}")

  return final_blurred_img

def clahe_equalization(image):
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  equalized_img = clahe.apply(image)
  return equalized_img

def otsu_thresholding(image):
  _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
  return binary_image

def canny_edge_detection(image, low_threshold=50, high_threshold=150):
  return cv2.Canny(image, low_threshold, high_threshold)

def find_extreme_corners(contours):
  '''Find the extreme corners of the image'''
  all_points = np.vstack(contours)
  top_left = all_points[np.argmin(all_points[:, :, 0] + all_points[:, :, 1])]
  bottom_right = all_points[np.argmax(all_points[:, :, 0] + all_points[:, :, 1])]
  top_right = all_points[np.argmax(all_points[:, :, 0] - all_points[:, :, 1])]
  bottom_left = all_points[np.argmin(all_points[:, :, 0] - all_points[:, :, 1])]
  return top_left[0], top_right[0], bottom_left[0], bottom_right[0]

def apply_perspective_transformation(image, corners):
  '''Apply perspective transformation to the image'''
  tl, tr, bl, br = corners
  width = int(max(np.linalg.norm(br - bl), np.linalg.norm(tr - tl)))
  height = int(max(np.linalg.norm(tr - br), np.linalg.norm(tl - bl)))

  dst_pts = np.array([
      [0, 0],
      [width - 1, 0],
      [0, height - 1],
      [width - 1, height - 1]
  ], dtype="float32")

  src_pts = np.array([tl, tr, bl, br], dtype="float32")

  M = cv2.getPerspectiveTransform(src_pts, dst_pts)
  warped = cv2.warpPerspective(image, M, (width, height))
  return warped

def automatic_warp_transformation(image, target_size=(800, 1000)):
  '''Automatic Cropping using Adaptive Warp Transformation'''
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  resized_image = resize_image(gray_image, target_size)
  brightened_image = logarithmic_transformation(resized_image)
  contrast_image = contrast_stretching(brightened_image)
  blurred_image = gaussian_blur(contrast_image, mode='Soft')
  binary_image = otsu_thresholding(blurred_image)
  edges = canny_edge_detection(binary_image)
  contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Getting Contours (Drawing Contours in image, useful for debugging)
  contour_image = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)
  cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

  corners = find_extreme_corners(contours)
  for corner in corners:
      cv2.circle(contour_image, tuple(corner), 5, (0, 0, 255), -1)

  warped_image = apply_perspective_transformation(resized_image, corners)
  print(f'Initial image {image.shape} processed to {warped_image.shape}')

  return warped_image

def automatic_warp_transformation_v2(image, target_size=(800, 1000)):
  '''Automatic Cropping using Adaptive Warp Transformation'''
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  resized_image = resize_image(gray_image, target_size)
  
  clahe = clahe_equalization(resized_image)
  log_img = logarithmic_transformation(clahe)
  contrast_img = contrast_stretching(log_img)
  blurred_img = gaussian_blur(contrast_img)
  binary_img = otsu_thresholding(blurred_img)
  edges = canny_edge_detection(binary_img)
  contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Getting Contours (Drawing Contours in image, useful for debugging)
  contour_image = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)
  cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

  corners = find_extreme_corners(contours)
  for corner in corners:
      cv2.circle(contour_image, tuple(corner), 5, (0, 0, 255), -1)

  warped_image = apply_perspective_transformation(resized_image, corners)
  print(f'Initial image {image.shape} processed to {warped_image.shape}')

  return warped_image

def image_uniformization(master_image, student_image):
  '''Precision Image Resizing'''
  master_shape = master_image.shape
  student_shape = student_image.shape

  master_height = master_shape[0]
  master_width = master_shape[1]

  student_height = student_shape[0]
  student_width = student_shape[1]

  min_height = min(master_height, student_height)
  min_width = min(master_width, student_width)

  resized_master = cv2.resize(master_image, (min_width, min_height))
  resized_student = cv2.resize(student_image, (min_width, min_height))

  print(f'master_key {master_image.shape} and student_answer {student_image.shape} uniformed to {resized_master.shape}')

  return resized_master, resized_student

def morph_open(image):
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
  eroded_img = cv2.erode(image, kernel, iterations = 1)
  dilated_img = cv2.dilate(eroded_img, kernel, iterations = 1)

  return dilated_img

def core_preprocessing(image):
  '''Core Preprocessing Module'''
  blurred_img = gaussian_blur(image, mode='Hard')
  contrast_img = contrast_stretching(blurred_img)
  log_img = logarithmic_transformation(contrast_img)
  binary_img = otsu_thresholding(log_img)
  opened_img = morph_open(binary_img)

  return opened_img

def core_preprocessing_v2(image):
  '''
  Core Preprocessing Module V2:
  - Uses CLAHE for lighting handling
  - Uses Adaptive Gaussian Blur to ensure optimal thresholding
  '''
  clahe_img = clahe_equalization(image)
  blurred_img = adaptive_gaussian_blur(clahe_img, desired_blur=100, max_iterations=100)
  contrast_img = contrast_stretching(blurred_img)
  log_img = logarithmic_transformation(contrast_img)
  binary_img = otsu_thresholding(log_img)
  opened_img = morph_open(binary_img)

  return opened_img

def draw_full_contours(contours, cont_image, radius = 7):
  '''Draw Full Circles'''
  for contour in contours:
    M = cv2.moments(contour)
    if M["m00"] != 0:
      cX = int(M["m10"] / M["m00"])
      cY = int(M["m01"] / M["m00"])
      # Draw a filled circle at the center of the contour
      cv2.circle(cont_image, (cX, cY), radius, (0, 255, 0), -1)

  return cont_image

def extract_and_draw_contours(image):
  contours, hierarchy = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

  unique_values = []
  for columns in image:
    for pixel in columns:
      if pixel not in unique_values:
        unique_values.append(pixel)

  contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
  contour_image = draw_full_contours(contours, contour_image)

  return contours, contour_image

def extract_and_draw_circle_contours(image):
  contours, hierarchy = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

  circle_contours = []
  contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

  # radius_list = []
  # contour_ar_list = []

  for contour in contours:
      # Approximate the enclosing circle for each contour
      (x, y), radius = cv2.minEnclosingCircle(contour)
      circle_area = np.pi * (radius ** 2)

      # if radius not in radius_list:
      #   radius_list.append(radius)

      # if contour_area not in contour_ar_list:
      #   contour_ar_list.append(contour_area)

      # Calculate the actual contour area
      contour_area = cv2.contourArea(contour)

      # Check if the contour area is approximately equal to the circle area
      # Tolerance range for being "circular"
      if radius < 5:
          if 0.6 <= contour_area / circle_area <= 1.4:
              circle_contours.append(contour)
      else:
          if 0.8 <= contour_area / circle_area <= 1.2:
              circle_contours.append(contour)

  # contour_image = cv2.drawContours(contour_image, circle_contours, -1, (0, 255, 0), thickness=2)
  contour_image = draw_full_contours(circle_contours, contour_image)

  return circle_contours, contour_image

def soft_morph_open(image):
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  eroded_img = cv2.erode(image, kernel, iterations = 1)
  dilated_img = cv2.dilate(eroded_img, kernel, iterations = 1)

  return dilated_img

def final_scoring(new_student, processed_student, master_contours):
  '''Final Score Calculation'''
  test_answer = processed_student.copy()
  # drawing the Answer Key to the Student's test answer, extracting the mistakes information
  check_answers = draw_full_contours(master_contours, test_answer)

  # open the image to remove noise
  final_sheet = soft_morph_open(check_answers)

  # fetching mistakes contours
  final_contours, img = extract_and_draw_circle_contours(final_sheet)
  
  # calculating mistakes and final score
  mistakes = len(final_contours)
  total_questions = len(master_contours)
  print(f'total_questions: {total_questions}, mistakes: {mistakes}')
  final_score = ((total_questions - mistakes) / total_questions) * 100
  print(f'final score: {final_score}')

  student_correction = cv2.cvtColor(new_student, cv2.COLOR_GRAY2BGR)
  student_correction = draw_full_contours(master_contours, student_correction)

  return final_score, student_correction

In [ ]:
raw_dir = 'C:/Users/vian8/Desktop/Tugas2/SNAPGRADE/inputs/cross_data_raw'
cropped_directory = 'C:/Users/vian8/Desktop/Tugas2/SNAPGRADE/inputs/cross_data_cropped'
os.makedirs(cropped_directory, exist_ok=True)

files = os.listdir(raw_dir)
image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

for i, filename in enumerate(image_files, start=1):
    old_path = os.path.join(raw_dir, filename)
    input_image = cv2.imread(old_path)

    if input_image is None:
        print(f"Error loading image: {old_path}")
        continue
    
    cropped_image = automatic_warp_transformation(input_image)

    file_extension = os.path.splitext(filename)[1]
    new_filename = f"{i}{file_extension}"
    new_path = os.path.join(cropped_directory, new_filename)

    cv2.imwrite(new_path, cropped_image)

print("Images have been transformed and saved successfully.")

Initial image (778, 1600, 3) processed to (465, 856)
Initial image (684, 1600, 3) processed to (410, 776)
Initial image (806, 1600, 3) processed to (483, 910)
Initial image (808, 1600, 3) processed to (471, 888)
Initial image (742, 1600, 3) processed to (445, 853)
Initial image (680, 1600, 3) processed to (415, 781)
Initial image (808, 1600, 3) processed to (486, 910)
Initial image (770, 1600, 3) processed to (460, 872)
Initial image (766, 1600, 3) processed to (459, 865)
Initial image (752, 1600, 3) processed to (453, 856)
Initial image (762, 1600, 3) processed to (453, 859)
Initial image (794, 1600, 3) processed to (479, 899)
Initial image (764, 1600, 3) processed to (453, 855)
Initial image (718, 1600, 3) processed to (428, 809)
Initial image (740, 1600, 3) processed to (438, 820)
Initial image (760, 1600, 3) processed to (451, 851)
Initial image (784, 1600, 3) processed to (478, 890)
Initial image (740, 1600, 3) processed to (446, 837)
Initial image (758, 1600, 3) processed to (461